In [1]:
import sys
import random

In [2]:
graphs_dir = './graphs_backup'
graph_name = 'graph_50_0.1.txt'
f = open(f'{graphs_dir}/{graph_name}', 'r')
lines = f.read().split('\n')
no_of_nodes = int(lines[0])
no_of_edges = int(lines[1])
weights = [float(lines[i].split(' ')[1]) for i in range(2, no_of_nodes + 2)]
edges = [(int(lines[i].split(' ')[0]), int(lines[i].split(' ')[1])) for i in range(no_of_nodes + 2, no_of_nodes + no_of_edges + 2)]

In [3]:
random.seed(1)

In [4]:
no_of_gens = 50
pop_size = 50
cross_prob = 0.5
mut_prob = 1 / no_of_nodes
init_pop = [[float(random.random() > 0.5) for n in range(no_of_nodes)] for i in range(pop_size)]
next_pop = []
best_slns = []
best_sln = (0, 0)

In [5]:
def validation2(indiv):
    for i in range(len(indiv)):
        if indiv[i] == 0.0:
            continue
        neighbor = [((i, j) in edges or (j, i) in edges) for j in range(i + 1, len(indiv)) if indiv[j] == 1.0]
        print(i, len(neighbor), neighbor)
        if any(neighbor):
            return False
    return True

In [6]:
def validation(indiv):
    for i in range(len(indiv)):
        if indiv[i] == 0.0:
            continue
        for j in range(i + 1, len(indiv)):
            if ((i, j) in edges or (j, i) in edges) and indiv[j] == 1.0:
                return False
    return True

In [7]:
def repair(indiv):
    for i in range(len(indiv)):
        if indiv[i] == 0.0:
            continue
        for j in range(i + 1, len(indiv)):
            if ((i, j) in edges or (j, i) in edges) and indiv[j] == 1.0:
                indiv[j] = 0.0
    return indiv

In [8]:
def repair_pop(population):
    repaired = []
    for individual in population:
        valid = validation(individual)
        if valid:
            repaired.append(individual)
        else:
            repaired.append(repair(individual))
    return repaired

In [9]:
def fitness(indiv):
    w = [weights[i] for i in range(len(indiv)) if indiv[i] == 1.0]
    return sum(w)

In [10]:
def tournament(population, fitness_values):
    mating_pool = []
    k = 2
    while len(mating_pool) < pop_size:
        indivs = random.sample(range(pop_size), k)
        max_indiv_weight = max([fitness_values[indiv] for indiv in indivs])
        max_fit_indiv = [indiv for indiv in indivs if fitness_values[indiv] == max_indiv_weight][0]
        mating_pool.append(population[max_fit_indiv])
    return mating_pool

In [11]:
def crossover(population):
    offsprings = []
    for i in range(0, len(population), 2):
        parent1 = population[i]
        parent2 = population[i + 1]
        if random.random() < cross_prob:
            offspring1 = []
            offspring2 = []
            for n in range(no_of_nodes):
                if random.random() < 0.5:
                    offspring1.append(parent1[n])
                    offspring2.append(parent2[n])
                else:
                    offspring1.append(parent2[n])
                    offspring2.append(parent1[n])
            offsprings.append(offspring1)
            offsprings.append(offspring2)
        else:
            offsprings.append(parent1)
            offsprings.append(parent2)
    return offsprings

In [12]:
def mutation(population):
    mutated = []
    k = 1
    for indiv in population:
        if random.random() < mut_prob:
            mutated_indiv = indiv
            gene = random.sample(range(no_of_nodes), k)
            for g in gene:
                mutated_indiv[g] = 1.0 - mutated_indiv[g]
            mutated.append(mutated_indiv)
        else:
            mutated.append(indiv)
    return mutated

In [13]:
next_pop = repair_pop(init_pop)
K = 20

for G in range(no_of_gens + 2):
    curr_pop = next_pop
    fitness_values = [fitness(indiv) for indiv in curr_pop]
    max_fitness = max(fitness_values)
    best_slns.append([indiv for indiv in curr_pop if fitness(indiv) == max_fitness][0])
    if G > best_sln[1] + K and max([fitness(sln) for sln in best_slns[-K:]]) <= best_sln[0]:
        print("best solution has not improved in the last K=20 generations since Gen. #" + best_sln[1])
        break
    if max_fitness > best_sln[0]:
        best_sln = (max_fitness, G)
    mating_pool = tournament(curr_pop, fitness_values)
    offsprings = crossover(mating_pool)
    mutated = mutation(offsprings)
    next_pop = repair_pop(mutated)

best solution has not improved in the last K=20 generations since Gen. # 30


In [14]:
for i, s in enumerate(best_slns):
    print(i, fitness(s))

0 8.69
1 9.37
2 9.29
3 9.29
4 9.29
5 9.6
6 9.73
7 9.74
8 9.84
9 10.29
10 10.29
11 10.29
12 10.29
13 10.29
14 10.29
15 10.29
16 10.29
17 10.29
18 10.29
19 10.29
20 10.29
21 10.29
22 10.29
23 10.29
24 10.29
25 10.29
26 10.29
27 10.29
28 10.29
29 10.29
30 10.37
31 10.37
32 10.37
33 10.37
34 10.37
35 10.37
36 10.37
37 10.37
38 10.37
39 10.37
40 9.51
41 10.37
42 10.37
43 10.37
44 10.37
45 10.37
46 10.37
47 10.37
48 10.37
49 10.37
50 10.37
51 10.37
